# <span style='font-family:rockwell'>  <span style='color:black'> ***Bradford Assay***

### <span style='font-family:rockwell'>  <span style='color:black'> **Data Processor**

In [4]:
import ipywidgets as widgets
import pandas as pd
import io 
import numpy as np

graph_btn = widgets.Button(description = "Display graph", button_style='info')

display(graph_btn)

out = widgets.Output()
display(out)

def test_button(self):
    with out:
        return print("the button works!")

graph_btn.on_click(test_button)


Button(button_style='info', description='Display graph', style=ButtonStyle())

Output()